In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
import torch
from transformers import AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import load_metric


# Process Data

## Load Data

In [ ]:
def create_dataset(file):

    with open(file, 'r') as f:
        lines = f.read()
    
    sentences = lines.split('\n\n')
    all_sent = []
    all_tag = []
    
    for sent in sentences:
        rows = sent.split("\n")
        words = []
        tags = []
        
        for r in rows[:-1]:
            w, t = r.split(" ")
            words.append(w)
            tags.append(t)
    
        all_sent.append(words[:])
        all_tag.append(tags[:])

    return pd.DataFrame({"tokens":all_sent, "ner_tags":all_tag})
    

In [ ]:
train_df = None

for i in [1,2,4,5]:
    file = "acl-out{}.conll".format(i)
    if train_df is None:
        train_df = create_dataset(file)
    else:
        train_df = pd.concat([train_df, create_dataset(file)], ignore_index=True)

val_df = create_dataset("acl-out3.conll")



In [ ]:
test_data = pd.read_csv('test_private.csv', index_col="id", skip_blank_lines=False)
inputs = test_data["input"].to_list()

for i in range(len(inputs)):
    if pd.isna(inputs[i]):
        inputs[i] = "END"

test_data["input"] = inputs
test_data['target'] = "O"

In [ ]:
test_texts = []
test_tags = []
test_words = test_data['input'].to_list()

sentence = []
for word in test_words[1:]:
    if word == "END":
        sentence.append(word)
        test_texts.append(sentence[:])
        test_tags.append(["O"]*len(sentence))
        sentence = []
    else:
        sentence.append(word)
sentence.append("END")

test_texts.append(sentence[:])
test_tags.append(["O"]*len(sentence))

test_df = pd.DataFrame({"tokens":test_texts, "ner_tags":test_tags})

## Tokenize Data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

labels = set()
for i, r in train_df.iterrows():    
    labels = labels.union(set(r['ner_tags']))

tag2id = {tag: id for id, tag in enumerate(labels)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
train_texts = train_df['tokens'].to_list()
val_texts = val_df['tokens'].to_list()
test_texts = test_df['tokens'].to_list()

train_tags = train_df['ner_tags'].to_list()
val_tags = val_df['ner_tags'].to_list()
test_tags = test_df['ner_tags'].to_list()

train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True, max_length=512)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True, max_length=512)
test_encodings = tokenizer(test_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True, max_length=512)


In [ ]:
def encode_tags(tags, encodings, test=False):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):

        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        if test:
            doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = tag2id["O"]
        else:
            doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels

        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)
test_labels = encode_tags(test_tags, test_encodings, True)


In [ ]:
class PaperDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
test_encodings.pop("offset_mapping")
train_dataset = PaperDataset(train_encodings, train_labels)
val_dataset = PaperDataset(val_encodings, val_labels)
test_dataset = PaperDataset(test_encodings, test_labels)


# Fine-tuning with trainer

## Setting Training Arguments

In [ ]:

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=25,              # total number of training epochs
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    evaluation_strategy = "epoch",
    # learning_rate=2e-5,
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = AutoModelForTokenClassification.from_pretrained("allenai/scibert_scivocab_uncased", num_labels=len(labels))


## Setting Metrics

In [ ]:
metric = load_metric("seqeval")
metric.compute(predictions=[labels], references=[labels])
label_list = list(labels)

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


## Setting Trainer

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    # data_collator=data_collator,
    eval_dataset=val_dataset,             # evaluation dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

## Train

In [ ]:
trainer.train()

## Eval

In [ ]:
trainer.evaluate()

## Test

In [ ]:
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=2)

predict = []
for prediction in predictions:
    level_p = []
    for p in prediction:
        if p != -100:
            level_p.append(id2tag[p])
    predict.append(level_p)

    

In [ ]:
# Set DocString
res = ["O"]
# Add "O" to sentence that exceed MAX_LENGTH of the pre-trained model, which is 512 tokens.
predict[676] += ["O"]*57
predict[466] += ["O"]*3

for i in range(len(predict)):
    res += predict[i][:]
    temp += len(test_df['tokens'][i])
# Remove the last END
res.pop()


## Output Result

In [ ]:
test_data['target'] = res
test_data.drop(columns=['input']).to_csv("test_res.csv")
test_data.to_csv("test_res_with_input.csv")